# Zero-Shot Event Classification

In [1]:
import json
import torch
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sentence_transformers import SentenceTransformer, util

## Implementing Simple Zero-Shot Classifier

Our approach in a nutshell:
* We use a sentence encoder from `sentence-transformers` to convert both label descriptions and texts to predict into embeddings that live in the same embedding space.
* At test time, we embed a new text and compare it to each label embedding via cosine similarity.
* We assign the label with the highest similarity to the item.
* Optionally, we define a minimum similarity threshold that a label needs to pass. If no label passes this threshold, we assign the "OTHER" class.


In [5]:
class ZeroShotClassifier:
    
    def __init__(self, model=None, threshold=None, null_label="OTHER"):
        self.model = model
        self.labels = []
        self.label_embeddings = None
        self.threshold = threshold
        self.null_label = null_label
    
    def train(self, labels, descriptions):
        self.labels = labels
        self.label_embeddings = model.encode(descriptions)
    
    def predict(self, input_texts=None, input_embeddings=None, output_scores=False):
        
        if input_embeddings is None:
            input_embeddings = self.model.encode(input_texts)
            
        S = util.pytorch_cos_sim(input_embeddings, self.label_embeddings)
        
        predicted_labels = []
        predicted_scores = []
        for i in range(input_embeddings.shape[0]):
            label_scores = S[i].tolist()
            scored = sorted(
                zip(self.labels, label_scores),
                key=lambda x: x[1],
                reverse=True
            )
            pred, score = scored[0]
            if self.threshold is not None and score < self.threshold:
                pred = self.null_label
                
            predicted_scores.append(scored)
            predicted_labels.append(pred)        
        
        if output_scores:
            return predicted_labels, predicted_scores
        else:
            return predicted_labels

## Preparing Data

In [6]:
import pandas as pd 
occ_df = pd.read_csv('occ-mgr-logs-2022-0124-0131.csv')
occ_df.head(5)

,_time,ENTRY
0,2022-01-28T02:06:00.000-0800,EOR.
1,2022-01-28T02:02:00.000-0800,M-Line Blanket established.
2,2022-01-28T01:52:00.000-0800,A-Line Blanket established.
3,2022-01-28T01:45:00.000-0800,"SA at M60 reports a sleeper on the platform, w..."
4,2022-01-28T01:38:00.000-0800,Core-Line Blanket established.


## Initializing Classifier

In [7]:
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
# Define sentence transformer model using CLS pooling
model_name = 'sentence-transformers/paraphrase-MiniLM-L6-v2'
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Define a list with sentences (1k - 100k sentences)
train_sentences = occ_df['ENTRY'].tolist()[:2000]

# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

# DataLoader to batch data
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

model.save('output/tsdae-model')
print("done")

When tie_encoder_decoder=True, the decoder_name_or_path will be invalid.
Some weights of BertLMHeadModel were not initialized from the model checkpoint at sentence-transformers/paraphrase-MiniLM-L6-v2 and are newly initialized: ['encoder.layer.4.crossattention.self.query.weight', 'encoder.layer.5.crossattention.self.query.weight', 'encoder.layer.2.crossattention.output.dense.weight', 'encoder.layer.3.crossattention.self.value.bias', 'encoder.layer.5.crossattention.self.value.weight', 'encoder.layer.4.crossattention.self.value.bias', 'encoder.layer.2.crossattention.self.query.bias', 'encoder.layer.5.crossattention.self.value.bias', 'encoder.layer.5.crossattention.output.dense.weight', 'encoder.layer.0.crossattention.output.LayerNorm.bias', 'encoder.layer.5.crossattention.self.key.bias', 'encoder.layer.4.crossattention.self.key.bias', 'encoder.layer.0.crossattention.self.value.bias', 'encoder.layer.1.crossattention.self.query.bias', 'encoder.layer.3.crossattention.self.query.weight', 'en

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

done


## Evaluating Zero-Shot Classifier

In [ ]:
import pandas as pd 

df = pd.read_csv('unlabeled.csv')[['Time', 'Log']].head(100)

df.head(5)

In [ ]:
my_classifier = ZeroShotClassifier(
    model=model,
    threshold=0.7,    
    null_label="OTHER"
)

my_classifier.train(
    labels=["Medical", "Police", 'Delays', 'Mechanical', 'Electrical'],
    descriptions=[
        "medical",
        "BPD activity",
        "delays late",
        "no ATO doors",
        'electrical'
    ]
)

pred = my_classifier.predict(
    df['Log'].tolist()
)
df['Pred'] = pred
df.head(20)